In [1]:
!pip install kmodes

Defaulting to user installation because normal site-packages is not writeable
You should consider upgrading via the '/usr/bin/python -m pip install --upgrade pip' command.


In [2]:
#Import modules
import numpy as np
import holidays
import pandas as pd
import seaborn as sns
import pickle
import time
import timeit


import matplotlib.pyplot as plt
plt.style.use('dark_background')
%matplotlib inline

import datetime
import math
from collections import Counter

#scipy
import scipy.stats as stats
from scipy import stats
from scipy.stats import chi2_contingency

#sklearn
import sklearn
from sklearn import ensemble
from sklearn import preprocessing
from sklearn.decomposition import PCA
from sklearn.ensemble import AdaBoostClassifier, BaggingClassifier, ExtraTreesClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_selection import SelectFromModel
from sklearn.impute import SimpleImputer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, f1_score, precision_score, log_loss, recall_score 
from sklearn.metrics import classification_report, confusion_matrix, roc_auc_score
from sklearn.model_selection import cross_val_score, GridSearchCV, train_test_split
from sklearn.preprocessing import LabelEncoder, OneHotEncoder, StandardScaler, OrdinalEncoder
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.utils import resample

#for clustering
from sklearn.cluster import KMeans
from sklearn.preprocessing import normalize
from sklearn.decomposition import PCA
from sklearn.cluster import MiniBatchKMeans
from sklearn.metrics import silhouette_score

#other learners
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from kmodes.kmodes import KModes

#imblearn
from imblearn.ensemble import BalancedBaggingClassifier
from imblearn.ensemble import EasyEnsembleClassifier
from imblearn.ensemble import BalancedRandomForestClassifier

#webscraping
import requests
from bs4 import BeautifulSoup
import re
import urllib
from IPython.core.display import HTML

#time series
import statsmodels.api as sm
from pylab import rcParams
import itertools
from statsmodels.tsa.stattools import adfuller
from statsmodels.tsa.stattools import acf, pacf
from statsmodels.tsa.seasonal import seasonal_decompose
from statsmodels.tsa.arima_model import ARIMA


#warning ignorer
import warnings
warnings.filterwarnings("ignore")

In [3]:
df = pd.read_csv('data/Road_Crashes_for_five_Years_-_Victoria.csv')

In [4]:
print('The dataset has {:3,d} samples and {} features'.format(*df.shape))

The dataset has 77,513 samples and 65 features


In [5]:
pd.set_option('max_column' ,None)

In [6]:
df

X            Y  OBJECTID   ACCIDENT_NO  \
0      2498217.170  2454870.252         1  T20140013514   
1      2502934.960  2423005.289         2  T20140013549   
2      2520154.039  2395621.674         3  T20140013561   
3      2473719.463  2402357.319         4  T20140013563   
4      2484073.998  2409123.682         5  T20140013567   
...            ...          ...       ...           ...   
77508  2508396.126  2431509.354     77509  T20200005929   
77509  2469786.824  2409460.190     77510  T20200010870   
77510  2443899.758  2372310.500     77511  T20200012822   
77511  2499271.806  2406893.351     77512  T20200013045   
77512  2501660.300  2417217.978     77513  T20200017444   

                      ABS_CODE ACCIDENT_STATUS ACCIDENT_DATE ACCIDENT_TIME  \
0      ABS to receive accident      Unfinished    2014/07/01      13:09:00   
1      ABS to receive accident        Finished    2014/07/02      00:01:00   
2      ABS to receive accident        Finished    2014/07/02      08:30:00   
3      ABS to receive accident        Finished    2014/07/02      10:08:00   
4      ABS to receive accident        Finished    2014/07/02      09:12:00   
...                        ...             ...           ...           ...   
77508  ABS to receive accident        Finished    2018/11/02      14:30:00   
77509  ABS to receive accident      Unfinished    2018/01/15      15:00:00   
77510  ABS to receive accident      Unfinished    2019/01/17      12:30:00   
77511  ABS to receive accident      Unfinished    2019/03/27      17:59:00   
77512  ABS to receive accident      Unfinished    2018/05/12      12:00:00   

      ALCOHOLTIME                      ACCIDENT_TYPE DAY_OF_WEEK  \
0              No             Collision with vehicle      Monday   
1             Yes             Collision with vehicle   Wednesday   
2              No   collision with some other object   Wednesday   
3              No             Collision with vehicle   Wednesday   
4              No             Collision with vehicle   Wednesday   
...           ...                                ...         ...   
77508          No             Collision with vehicle    Thursday   
77509          No             Collision with vehicle      Sunday   
77510          No                  Struck Pedestrian   Wednesday   
77511          No  No collision and no object struck     Tuesday   
77512          No             Collision with vehicle      Friday   

                                                DCA_CODE HIT_RUN_FLAG  \
0                                          RIGHT THROUGH           No   
1                                                 U TURN           No   
2                           STRUCK OBJECT ON CARRIAGEWAY           No   
3                      CROSS TRAFFIC(INTERSECTIONS ONLY)          Yes   
4                                          RIGHT THROUGH           No   
...                                                  ...          ...   
77508                    REAR END(VEHICLES IN SAME LANE)           No   
77509                  CROSS TRAFFIC(INTERSECTIONS ONLY)           No   
77510                                PARKED CAR RUN AWAY           No   
77511  UNKNOWN-NO DETAILS ON MANOEUVRES OF ROAD-USERS...           No   
77512                  CROSS TRAFFIC(INTERSECTIONS ONLY)           No   

             LIGHT_CONDITION POLICE_ATTEND        ROAD_GEOMETRY  \
0                        Day           Yes              Unknown   
1      Dark Street lights on           Yes  Not at intersection   
2                  Dusk/Dawn           Yes  Not at intersection   
3                       Unk.            No   Cross intersection   
4                        Day           Yes   Cross intersection   
...                      ...           ...                  ...   
77508                    Day            No       T intersection   
77509                   Unk.     Not known              Unknown   
77510                    Day            No              Unknown   
77511  

In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 77513 entries, 0 to 77512
Data columns (total 65 columns):
X                    77513 non-null float64
Y                    77513 non-null float64
OBJECTID             77513 non-null int64
ACCIDENT_NO          77513 non-null object
ABS_CODE             77513 non-null object
ACCIDENT_STATUS      77513 non-null object
ACCIDENT_DATE        77513 non-null object
ACCIDENT_TIME        77513 non-null object
ALCOHOLTIME          77513 non-null object
ACCIDENT_TYPE        77513 non-null object
DAY_OF_WEEK          75153 non-null object
DCA_CODE             77513 non-null object
HIT_RUN_FLAG         77513 non-null object
LIGHT_CONDITION      77513 non-null object
POLICE_ATTEND        77513 non-null object
ROAD_GEOMETRY        77513 non-null object
SEVERITY             77512 non-null object
SPEED_ZONE           77513 non-null object
RUN_OFFROAD          77513 non-null object
NODE_ID              77513 non-null int64
LONGITUDE            76407 non-

In [8]:
pd.set_option('max_row' ,None)

In [9]:
df.describe().T

count          mean            std           min  \
X                  77513.0  2.497548e+06   74449.157609  2.128750e+06   
Y                  77513.0  2.421967e+06   64226.057869  2.274524e+06   
OBJECTID           77513.0  3.875700e+04   22376.220045  1.000000e+00   
NODE_ID            77513.0  2.277329e+05  136413.297855  4.000000e+00   
LONGITUDE          76407.0  1.396826e+02      27.293030 -1.000000e+00   
LATITUDE           76407.0 -3.646759e+01       6.636517 -3.902399e+01   
VICGRID_X          77513.0  2.497548e+06   74449.157609  2.128750e+06   
VICGRID_Y          77513.0  2.421967e+06   64226.057869  2.274524e+06   
TOTAL_PERSONS      77512.0  2.346166e+00       1.499449  1.000000e+00   
INJ_OR_FATAL       77512.0  1.269365e+00       0.725633  0.000000e+00   
FATALITY           77512.0  2.171277e-02       0.155758  0.000000e+00   
SERIOUSINJURY      77512.0  4.691145e-01       0.661838  0.000000e+00   
OTHERINJURY        77512.0  7.785375e-01       0.753042  0.000000e+00   
NONINJURED         77512.0  1.076672e+00       1.290616  0.000000e+00   
MALES              77512.0  1.298380e+00       0.984148  0.000000e+00   
FEMALES            77512.0  9.253793e-01       1.053217  0.000000e+00   
BICYCLIST          77512.0  9.465631e-02       0.303772  0.000000e+00   
PASSENGER          77512.0  4.878986e-01       1.142450  0.000000e+00   
DRIVER             77512.0  1.431314e+00       0.904407  0.000000e+00   
PEDESTRIAN         77512.0  1.121375e-01       0.384467  0.000000e+00   
PILLION            77512.0  5.341108e-03       0.073768  0.000000e+00   
MOTORIST           77512.0  1.572531e-01       0.373557  0.000000e+00   
UNKNOWN            77512.0  5.753948e-02       0.410845  0.000000e+00   
PED_CYCLIST_5_12   77512.0  9.353390e-03       0.104239  0.000000e+00   
PED_CYCLIST_13_18  77512.0  1.545567e-02       0.129181  0.000000e+00   
OLD_PEDESTRIAN     77512.0  2.382857e-02       0.156359  0.000000e+00   
OLD_DRIVER         77512.0  6.948602e-02       0.260099  0.000000e+00   
YOUNG_DRIVER       77512.0  2.891423e-01       0.511404  0.000000e+00   
UNLICENCSED        77512.0  3.387863e-02       0.182197  0.000000e+00   
NO_OF_VEHICLES     77508.0  1.785880e+00       0.767924  1.000000e+00   
HEAVYVEHICLE       77508.0  4.564690e-02       0.220678  0.000000e+00   
PASSENGERVEHICLE   77508.0  1.405197e+00       0.932913  0.000000e+00   
MOTORCYCLE         77508.0  1.578805e-01       0.374821  0.000000e+00   
PUBLICVEHICLE      77508.0  1.232131e-02       0.111364  0.000000e+00   

                            25%           50%           75%           max  
X                  2.485326e+06  2.501483e+06  2.518924e+06  2.920148e+06  
Y                  2.393837e+06  2.409527e+06  2.423384e+06  2.820900e+06  
OBJECTID           1.937900e+04  3.875700e+04  5.813500e+04  7.751300e+04  
NODE_ID            5.130100e+04  2.933300e+05  3.212700e+05  6.764220e+05  
LONGITUDE          1.447749e+02  1.450058e+02  1.452048e+02  1.497575e+02  
LATITUDE          -3.794697e+01 -3.781017e+01 -3.766746e+01 -1.000000e+00  
VICGRID_X          2.485326e+06  2.501483e+06  2.518924e+06  2.920148e+06  
VICGRID_Y          2.393837e+06  2.409527e+06  2.423384e+06  2.820900e+06  
TOTAL_PERSONS      2.000000e+00  2.000000e+00  3.000000e+00  8.900000e+01  
INJ_OR_FATAL       1.000000e+00  1.000000e+00  1.000000e+00  4.100000e+01  
FATALITY           0.000000e+00  0.000000e+00  0.000000e+00  6.000000e+00  
SERIOUSINJURY      0.000000e+00  0.000000e+00  1.000000e+00  1.800000e+01  
OTHERINJURY        0.000000e+00  1.000000e+00  1.000000e+00  2.500000e+01  
NONINJURED         0.000000e+00  1.000000e+00  1.000000e+00  8.700000e+01  
MALES              1.000000e+00  1.000000e+00  2.000000e+00  3.000000e+01  
FEMALES            0.000000e+00  1.000000e+00  1.000000e+00  5.100000e+01  
BICYCLIST          0.000000e+00  0.000000e+00  0.000000e+00  8.000000e+00  
PASSENGER          0.000000e+00  0.000000e+00  1.000000e+00  8.500000e+01  
DRIVER             1.

In [10]:
#let's take a closer look at nulls within columns
null_columns=df.columns[df.isnull().any()]
df[null_columns].isnull().sum() 

DAY_OF_WEEK           2360
SEVERITY                 1
LONGITUDE             1106
LATITUDE              1106
NODE_TYPE               89
TOTAL_PERSONS            1
INJ_OR_FATAL             1
FATALITY                 1
SERIOUSINJURY            1
OTHERINJURY              1
NONINJURED               1
MALES                    1
FEMALES                  1
BICYCLIST                1
PASSENGER                1
DRIVER                   1
PEDESTRIAN               1
PILLION                  1
MOTORIST                 1
UNKNOWN                  1
PED_CYCLIST_5_12         1
PED_CYCLIST_13_18        1
OLD_PEDESTRIAN           1
OLD_DRIVER               1
YOUNG_DRIVER             1
ALCOHOL_RELATED          1
UNLICENCSED              1
NO_OF_VEHICLES           5
HEAVYVEHICLE             5
PASSENGERVEHICLE         5
MOTORCYCLE               5
PUBLICVEHICLE            5
REGION_NAME_ALL          1
SRNS                 54142
SRNS_ALL             54142
RMA                   5452
RMA_ALL               5452
D

# Nulls and Outliers

In [11]:
df.isnull().sum().sort_values(ascending=False)/df.shape[0]*100

SRNS_ALL             69.848929
SRNS                 69.848929
DIVIDED               7.033659
RMA_ALL               7.033659
RMA                   7.033659
DIVIDED_ALL           7.033659
DAY_OF_WEEK           3.044651
LONGITUDE             1.426857
LATITUDE              1.426857
NODE_TYPE             0.114819
PASSENGERVEHICLE      0.006451
NO_OF_VEHICLES        0.006451
PUBLICVEHICLE         0.006451
MOTORCYCLE            0.006451
HEAVYVEHICLE          0.006451
STAT_DIV_NAME         0.002580
DRIVER                0.001290
TOTAL_PERSONS         0.001290
INJ_OR_FATAL          0.001290
FATALITY              0.001290
SERIOUSINJURY         0.001290
OTHERINJURY           0.001290
SEVERITY              0.001290
MALES                 0.001290
FEMALES               0.001290
BICYCLIST             0.001290
PASSENGER             0.001290
NONINJURED            0.001290
PEDESTRIAN            0.001290
MOTORIST              0.001290
UNKNOWN               0.001290
PED_CYCLIST_5_12      0.001290
PED_CYCL

In [12]:
#df.dropna(how='any',inplace=True)

In [13]:
df.shape

(77513, 65)

In [14]:
df['SRNS_ALL'].value_counts()/df.shape[0]*100

C        13.792525
M         7.696773
A         3.930954
B         3.532311
A,C       0.203837
M,C       0.197386
C,A       0.180615
B,C       0.174164
C,M       0.157393
C,B       0.123850
A,B       0.046444
B,A       0.029672
M,A       0.020642
B,M       0.014191
A,M       0.011611
A,C,B     0.011611
B,A,C     0.009031
M,B       0.009031
B,C,M     0.005160
A,M,C     0.002580
C,B,M     0.001290
Name: SRNS_ALL, dtype: float64

In [15]:
df['SRNS_ALL'].fillna(method='ffill', inplace=True)

In [16]:
df['SRNS'].value_counts()/df.shape[0]*100

C    13.792525
M     8.116058
A     4.412163
B     3.830325
Name: SRNS, dtype: float64

In [17]:
df['SRNS'].fillna(method='ffill', inplace=True)

In [18]:
df['DIVIDED'].value_counts()/df.shape[0]*100

Undivided    59.885439
Divided      33.080903
Name: DIVIDED, dtype: float64

In [19]:
df['DIVIDED'].fillna(method='ffill', inplace=True)

In [20]:
df['RMA_ALL'].value_counts()/df.shape[0]*100

Local Road                                    33.253777
Arterial Other                                19.515436
Arterial Other,Local Road                     11.976056
Arterial Highway                               9.066866
Freeway                                        5.938359
Local Road,Arterial Highway                    4.476668
Local Road,Arterial Other                      2.965954
Arterial Highway,Local Road                    1.531356
Arterial Other,Arterial Highway                1.260434
Arterial Highway,Arterial Other                0.915975
Freeway,Arterial Other                         0.643763
Arterial Other,Local Road,Arterial Highway     0.336718
Arterial Highway,Arterial Other,Local Road     0.134171
Arterial Highway,Freeway                       0.129011
Freeway,Local Road                             0.127721
Arterial Other,Freeway                         0.119980
Freeway,Arterial Highway                       0.103208
Local Road,Arterial Highway,Arterial Other     0

In [21]:
df['RMA_ALL'].fillna(method='ffill', inplace=True)

In [22]:
df['RMA'].value_counts()/df.shape[0]*100

Arterial Other      34.467767
Local Road          33.253777
Arterial Highway    17.864100
Freeway              7.354895
Non Arterial         0.025802
Name: RMA, dtype: float64

In [23]:
df['RMA'].fillna(method='ffill', inplace=True)

In [24]:
df['DIVIDED_ALL'].value_counts()/df.shape[0]*100

Undivided            59.885439
Divided              18.069227
Undivided,Divided     8.486318
Divided,Undivided     6.525357
Name: DIVIDED_ALL, dtype: float64

In [25]:
df['DIVIDED_ALL'].fillna(method='ffill', inplace=True)

In [26]:
df['DAY_OF_WEEK'].value_counts()/df.shape[0]*100

Friday       15.251635
Thursday     15.063280
Wednesday    14.687859
Tuesday      14.233741
Monday       13.620941
Sunday       13.120380
Saturday     10.977513
Name: DAY_OF_WEEK, dtype: float64

In [27]:
df['DAY_OF_WEEK'].fillna(method='ffill', inplace=True)

In [28]:
df['SEVERITY'].value_counts()/df.shape[0]*100

Other injury accident      58.003174
Serious injury accident    39.937817
Fatal accident              2.053849
Non injury accident         0.003870
Name: SEVERITY, dtype: float64

In [29]:
df['SEVERITY'].fillna(method='ffill', inplace=True)

In [30]:
df['LONGITUDE'].fillna(method='ffill', inplace=True)

In [31]:
df['LATITUDE'].fillna(method='ffill', inplace=True)

In [32]:
#### Check nulls for again
df.isnull().sum().sort_values(ascending=False)/df.shape[0]*100

NODE_TYPE            0.114819
PUBLICVEHICLE        0.006451
MOTORCYCLE           0.006451
NO_OF_VEHICLES       0.006451
HEAVYVEHICLE         0.006451
PASSENGERVEHICLE     0.006451
STAT_DIV_NAME        0.002580
OTHERINJURY          0.001290
MALES                0.001290
FEMALES              0.001290
BICYCLIST            0.001290
PED_CYCLIST_5_12     0.001290
SERIOUSINJURY        0.001290
FATALITY             0.001290
INJ_OR_FATAL         0.001290
TOTAL_PERSONS        0.001290
PASSENGER            0.001290
DRIVER               0.001290
PEDESTRIAN           0.001290
PILLION              0.001290
MOTORIST             0.001290
UNKNOWN              0.001290
NONINJURED           0.001290
PED_CYCLIST_13_18    0.001290
OLD_DRIVER           0.001290
YOUNG_DRIVER         0.001290
ALCOHOL_RELATED      0.001290
UNLICENCSED          0.001290
OLD_PEDESTRIAN       0.001290
REGION_NAME_ALL      0.001290
ACCIDENT_TYPE        0.000000
ROAD_GEOMETRY        0.000000
POLICE_ATTEND        0.000000
LIGHT_COND

In [33]:
#drop the remaining nulls that are <1%
df.dropna(inplace=True)

#last check
df.isnull().sum().sort_values(ascending=False)/df.shape[0]*100

STAT_DIV_NAME        0.0
OTHERINJURY          0.0
FATALITY             0.0
INJ_OR_FATAL         0.0
TOTAL_PERSONS        0.0
VICGRID_Y            0.0
VICGRID_X            0.0
REGION_NAME          0.0
LGA_NAME             0.0
NODE_TYPE            0.0
LATITUDE             0.0
LONGITUDE            0.0
NODE_ID              0.0
RUN_OFFROAD          0.0
SPEED_ZONE           0.0
SEVERITY             0.0
ROAD_GEOMETRY        0.0
POLICE_ATTEND        0.0
LIGHT_CONDITION      0.0
HIT_RUN_FLAG         0.0
DCA_CODE             0.0
DAY_OF_WEEK          0.0
ACCIDENT_TYPE        0.0
ALCOHOLTIME          0.0
ACCIDENT_TIME        0.0
ACCIDENT_DATE        0.0
ACCIDENT_STATUS      0.0
ABS_CODE             0.0
ACCIDENT_NO          0.0
OBJECTID             0.0
Y                    0.0
SERIOUSINJURY        0.0
NONINJURED           0.0
DIVIDED_ALL          0.0
MALES                0.0
DIVIDED              0.0
RMA_ALL              0.0
RMA                  0.0
SRNS_ALL             0.0
SRNS                 0.0


In [34]:
df.shape

(77416, 65)

In [35]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 77416 entries, 0 to 77512
Data columns (total 65 columns):
X                    77416 non-null float64
Y                    77416 non-null float64
OBJECTID             77416 non-null int64
ACCIDENT_NO          77416 non-null object
ABS_CODE             77416 non-null object
ACCIDENT_STATUS      77416 non-null object
ACCIDENT_DATE        77416 non-null object
ACCIDENT_TIME        77416 non-null object
ALCOHOLTIME          77416 non-null object
ACCIDENT_TYPE        77416 non-null object
DAY_OF_WEEK          77416 non-null object
DCA_CODE             77416 non-null object
HIT_RUN_FLAG         77416 non-null object
LIGHT_CONDITION      77416 non-null object
POLICE_ATTEND        77416 non-null object
ROAD_GEOMETRY        77416 non-null object
SEVERITY             77416 non-null object
SPEED_ZONE           77416 non-null object
RUN_OFFROAD          77416 non-null object
NODE_ID              77416 non-null int64
LONGITUDE            77416 non-

# Feature Manipulation Creation and Engineering

In [40]:
# #unique values
df.nunique().sort_values(ascending=False)

OBJECTID             77416
ACCIDENT_NO          77416
LATITUDE             72326
LONGITUDE            72164
NODE_ID              61281
Y                    61087
VICGRID_Y            61087
X                    61081
VICGRID_X            61081
ACCIDENT_DATE         1826
ACCIDENT_TIME         1424
LGA_NAME_ALL           223
LGA_NAME                91
DCA_CODE                81
RMA_ALL                 46
TOTAL_PERSONS           38
PASSENGER               33
NONINJURED              33
FEMALES                 23
INJ_OR_FATAL            22
SRNS_ALL                21
MALES                   20
REGION_NAME_ALL         19
DEG_URBAN_ALL           18
OTHERINJURY             18
UNKNOWN                 16
PASSENGERVEHICLE        15
SERIOUSINJURY           14
NO_OF_VEHICLES          14
DRIVER                  14
SPEED_ZONE              13
PEDESTRIAN              11
ACCIDENT_TYPE            9
ROAD_GEOMETRY            9
BICYCLIST                8
REGION_NAME              8
LIGHT_CONDITION          7
D

In [41]:
df['ACCIDENT_DATE'] = pd.to_datetime(df['ACCIDENT_DATE'])

In [42]:
df['month'] = df ['ACCIDENT_DATE'].apply(lambda time: time.month)

In [45]:
df.head()

X            Y  OBJECTID   ACCIDENT_NO                 ABS_CODE  \
0  2498217.170  2454870.252         1  T20140013514  ABS to receive accident   
1  2502934.960  2423005.289         2  T20140013549  ABS to receive accident   
2  2520154.039  2395621.674         3  T20140013561  ABS to receive accident   
3  2473719.463  2402357.319         4  T20140013563  ABS to receive accident   
4  2484073.998  2409123.682         5  T20140013567  ABS to receive accident   

  ACCIDENT_STATUS ACCIDENT_DATE ACCIDENT_TIME ALCOHOLTIME  \
0      Unfinished    2014-07-01      13:09:00          No   
1        Finished    2014-07-02      00:01:00         Yes   
2        Finished    2014-07-02      08:30:00          No   
3        Finished    2014-07-02      10:08:00          No   
4        Finished    2014-07-02      09:12:00          No   

                      ACCIDENT_TYPE DAY_OF_WEEK  \
0            Collision with vehicle      Monday   
1            Collision with vehicle   Wednesday   
2  collision with some other object   Wednesday   
3            Collision with vehicle   Wednesday   
4            Collision with vehicle   Wednesday   

                            DCA_CODE HIT_RUN_FLAG        LIGHT_CONDITION  \
0                      RIGHT THROUGH           No                    Day   
1                             U TURN           No  Dark Street lights on   
2       STRUCK OBJECT ON CARRIAGEWAY           No              Dusk/Dawn   
3  CROSS TRAFFIC(INTERSECTIONS ONLY)          Yes                   Unk.   
4                      RIGHT THROUGH           No                    Day   

  POLICE_ATTEND        ROAD_GEOMETRY               SEVERITY SPEED_ZONE  \
0           Yes              Unknown  Other injury accident   60 km/hr   
1           Yes  Not at intersection  Other injury accident   60 km/hr   
2           Yes  Not at intersection  Other injury accident   50 km/hr   
3            No   Cross intersection  Other injury accident  Not known   
4           Yes   Cross intersection  Other injury accident   60 km/hr   

  RUN_OFFROAD  NODE_ID   LONGITUDE   LATITUDE         NODE_TYPE    LGA_NAME  \
0          No    49465  144.979861 -37.406699      Intersection    MITCHELL   
1          No   272595   -1.000000  -1.000000  Non-Intersection  WHITTLESEA   
2          No   273609   -1.000000  -1.000000  Non-Intersection        KNOX   
3          No    44382   -1.000000  -1.000000      Intersection     WYNDHAM   
4          No    38531   -1.000000  -1.000000      Intersection     WYNDHAM   

                      REGION_NAME    VICGRID_X    VICGRID_Y  TOTAL_PERSONS  \
0                 NORTHERN REGION  2498217.170  2454870.252            4.0   
1  METROPOLITAN NORTH WEST REGION  2502934.960  2423005.289            4.0   
2  METROPOLITAN SOUTH EAST REGION  2520154.039  2395621.674            1.0   
3  METROPOLITAN NORTH WEST REGION  2473719.463  2402357.319            3.0   
4  METROPOLITAN NORTH WEST REGION  2484073.998  2409123.682            2.0   

   INJ_OR_FATAL  FATALITY  SERIOUSINJURY  OTHERINJURY  NONINJURED  MALES  \
0           1.0       0.0            0.0          1.0         3.0    1.0   
1           3.0       0.0            0.0          3.0         1.0    0.0   
2           1.0       0.0            0.0          1.0         0.0    0.0   
3           1.0       0.0            0.0          1.0         2.0    2.0   
4           1.0       0.0            0.0          1.0         1.0    2.0   

   FEMALES  BICYCLIST  PASSENGER  DRIVER  PEDESTRIAN  PILLION  MOTORIST  \
0      3.0        0.0        2.0     2.0         0.0      0.0       0.0   
1      3.0        0.0        2.0     2.0         0.0      0.0       0.0   
2      1.0        0.0        0.0     1.0         0.0      0.0       0.0   
3      1.0        0.0        1.0     2.0         0.0      0.0       0.0   
4      0.0        0.0        0.0     2.0         0.0      0.0       0.0   

   UNKNOWN  PED_CYCLIST_5_12  PED_CYCLIST_13_18  OLD_PEDESTRIAN  OLD_DRIVER  \
0      0.0               

In [44]:
#creating a weekend feature that includes Friday-Sunday
df['weekend']= np.where(df['DAY_OF_WEEK'].isin(['Friday', 'Saturday', 'Sunday']), 1, 0)

In [46]:
#create time of day feature with Morning Rush, Day, Noon Rush, Afternoon, After Work Rush, Night

#time of day dictionary
timeofdaygroups = {1: "Morning Rush (6-10)",
                   2: "Day (10-12)",
                   3: "Lunch Rush (12-14)",
                   4: "Afternoon (14-16)",
                   5: "After Work Rush (16-18)",
                   6: "Evening (18-22)",
                   7: "Night (22-6)"}

In [47]:
#pull time data and create hour column
df['hour'] = df['ACCIDENT_TIME'].str[0:2]
 
#convert to numeric    
df['hour'] =  pd.to_numeric(df['hour'])

#convert to integer
df['hour'] = df['hour'].astype('int')

In [48]:
#create time_of_day grouping

def daygroup(hour):
    if hour >= 6 and hour < 10:
        return "1"
    elif hour >= 10 and hour < 12:
        return "2"
    elif hour >= 12 and hour < 14:
        return "3"
    elif hour >= 14 and hour < 16:
        return "4"
    elif hour >= 16 and hour < 18:
        return "5"
    elif hour >= 18 and hour < 22:
        return "6"
    else:
        return "7"
    

In [49]:
#apply function   
#time of day function
df['time_of_day']= df['hour'].apply(daygroup)   

In [50]:
df[['weekend','DAY_OF_WEEK','ACCIDENT_TIME', 'time_of_day']].tail(10)

weekend DAY_OF_WEEK ACCIDENT_TIME time_of_day
77503        0   Wednesday      15:20:00           4
77504        0    Thursday      15:15:00           4
77505        0   Wednesday      07:45:00           1
77506        1      Sunday      01:00:00           7
77507        1      Sunday      15:30:00           4
77508        0    Thursday      14:30:00           4
77509        1      Sunday      15:00:00           4
77510        0   Wednesday      12:30:00           3
77511        0     Tuesday      17:59:00           5
77512        1      Friday      12:00:00           3

In [51]:
#Put above pickle in next full run
#create new column for Machine Learning with Injury and Serious/Fatal
df['accident_seriousness'] = df['SEVERITY']
df['accident_seriousness'] = df['accident_seriousness'].replace(to_replace="Other injury accident", 
                                                                value="Injury")
df['accident_seriousness'] = df['accident_seriousness'].replace(to_replace="Non injury accident",
                                                                value="Injury")

df['accident_seriousness'] = df['accident_seriousness'].replace(to_replace="Serious injury accident", 
                                                                value="Serious/Fatal")
df['accident_seriousness'] = df['accident_seriousness'].replace(to_replace="Fatal accident", 
                                                                value="Serious/Fatal")
df.shape

(77416, 70)

In [52]:
df.accident_seriousness.value_counts()

Injury           44909
Serious/Fatal    32507
Name: accident_seriousness, dtype: int64

In [53]:
df.head()

X            Y  OBJECTID   ACCIDENT_NO                 ABS_CODE  \
0  2498217.170  2454870.252         1  T20140013514  ABS to receive accident   
1  2502934.960  2423005.289         2  T20140013549  ABS to receive accident   
2  2520154.039  2395621.674         3  T20140013561  ABS to receive accident   
3  2473719.463  2402357.319         4  T20140013563  ABS to receive accident   
4  2484073.998  2409123.682         5  T20140013567  ABS to receive accident   

  ACCIDENT_STATUS ACCIDENT_DATE ACCIDENT_TIME ALCOHOLTIME  \
0      Unfinished    2014-07-01      13:09:00          No   
1        Finished    2014-07-02      00:01:00         Yes   
2        Finished    2014-07-02      08:30:00          No   
3        Finished    2014-07-02      10:08:00          No   
4        Finished    2014-07-02      09:12:00          No   

                      ACCIDENT_TYPE DAY_OF_WEEK  \
0            Collision with vehicle      Monday   
1            Collision with vehicle   Wednesday   
2  collision with some other object   Wednesday   
3            Collision with vehicle   Wednesday   
4            Collision with vehicle   Wednesday   

                            DCA_CODE HIT_RUN_FLAG        LIGHT_CONDITION  \
0                      RIGHT THROUGH           No                    Day   
1                             U TURN           No  Dark Street lights on   
2       STRUCK OBJECT ON CARRIAGEWAY           No              Dusk/Dawn   
3  CROSS TRAFFIC(INTERSECTIONS ONLY)          Yes                   Unk.   
4                      RIGHT THROUGH           No                    Day   

  POLICE_ATTEND        ROAD_GEOMETRY               SEVERITY SPEED_ZONE  \
0           Yes              Unknown  Other injury accident   60 km/hr   
1           Yes  Not at intersection  Other injury accident   60 km/hr   
2           Yes  Not at intersection  Other injury accident   50 km/hr   
3            No   Cross intersection  Other injury accident  Not known   
4           Yes   Cross intersection  Other injury accident   60 km/hr   

  RUN_OFFROAD  NODE_ID   LONGITUDE   LATITUDE         NODE_TYPE    LGA_NAME  \
0          No    49465  144.979861 -37.406699      Intersection    MITCHELL   
1          No   272595   -1.000000  -1.000000  Non-Intersection  WHITTLESEA   
2          No   273609   -1.000000  -1.000000  Non-Intersection        KNOX   
3          No    44382   -1.000000  -1.000000      Intersection     WYNDHAM   
4          No    38531   -1.000000  -1.000000      Intersection     WYNDHAM   

                      REGION_NAME    VICGRID_X    VICGRID_Y  TOTAL_PERSONS  \
0                 NORTHERN REGION  2498217.170  2454870.252            4.0   
1  METROPOLITAN NORTH WEST REGION  2502934.960  2423005.289            4.0   
2  METROPOLITAN SOUTH EAST REGION  2520154.039  2395621.674            1.0   
3  METROPOLITAN NORTH WEST REGION  2473719.463  2402357.319            3.0   
4  METROPOLITAN NORTH WEST REGION  2484073.998  2409123.682            2.0   

   INJ_OR_FATAL  FATALITY  SERIOUSINJURY  OTHERINJURY  NONINJURED  MALES  \
0           1.0       0.0            0.0          1.0         3.0    1.0   
1           3.0       0.0            0.0          3.0         1.0    0.0   
2           1.0       0.0            0.0          1.0         0.0    0.0   
3           1.0       0.0            0.0          1.0         2.0    2.0   
4           1.0       0.0            0.0          1.0         1.0    2.0   

   FEMALES  BICYCLIST  PASSENGER  DRIVER  PEDESTRIAN  PILLION  MOTORIST  \
0      3.0        0.0        2.0     2.0         0.0      0.0       0.0   
1      3.0        0.0        2.0     2.0         0.0      0.0       0.0   
2      1.0        0.0        0.0     1.0         0.0      0.0       0.0   
3      1.0        0.0        1.0     2.0         0.0      0.0       0.0   
4      0.0        0.0        0.0     2.0         0.0      0.0       0.0   

   UNKNOWN  PED_CYCLIST_5_12  PED_CYCLIST_13_18  OLD_PEDESTRIAN  OLD_DRIVER  \
0      0.0               

In [59]:
# Create csv of cleaned data to use later
df.to_csv('Cleaned_dataset.csv')